In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import constants as const
import scipy.special as special
import bilby
import H1 as get_interf

In [ ]:
#### Define time of event in GPS
time_of_event=1126259462.4
#### Get the data and PSD
H1 = get_interf.get_H1(time_of_event=1126259462.4, post_trigger_duration=2, duration=4, psd_duration_multi=32)

In [ ]:
#### Our model
pc = (const.pc).value
Gpc = pow(10,9)*pc
M_sun = (const.M_sun).value
G = (const.G).value
c = (const.c).value
#### Defining the model
def echo(frequency_array, R, d, M, D, wr, wi, alpha, t_0, phi):

    
    w = 2 * np.pi * frequency_array
    
    
    Zp = np.sqrt(np.pi/2) * (M/D) * (M_sun/Gpc) * np.float64(G/(c**2)) * (np.exp(1j*(w-(1j*wi)/(M*(M_sun*(G/(c**3)))))*t_0)*(1+R)*special.gamma(1-((1j*w)/(alpha/(M*(M_sun*(G/(c**3)))))))*(((wr/(M*(M_sun*(G/(c**3)))))*np.sin(((wr/(M*(M_sun*(G/(c**3)))))*t_0)+phi)+((1j*(w-(1j*wi)/(M*(M_sun*(G/(c**3))))))*np.cos((wr/(M*(M_sun*(G/(c**3))))*t_0)+phi))))/
                                                                          ((((w-(1j*wi)/(M*(M_sun*(G/(c**3)))))*(w-(1j*wi)/(M*(M_sun*(G/(c**3))))))-((wr/(M*(M_sun*(G/(c**3)))))**2))*((np.pi*special.gamma(1-((1j*w)/(alpha/(M*(M_sun*(G/(c**3))))))))+(np.exp(2j*d*(M*(M_sun*(G/(c**3))))*w)*R*np.cosh((np.pi*wr)/alpha)*special.gamma((1/2)-(1j*(w+(wr/(M*(M_sun*(G/(c**3))))))/(alpha/(M*(M_sun*(G/(c**3)))))))*special.gamma((1/2)-(1j*(w-(wr/(M*(M_sun*(G/(c**3))))))/(alpha/(M*(M_sun*(G/(c**3)))))))*special.gamma(1+((1j*w)/(alpha/(M*(M_sun*(G/(c**3)))))))))))

    cross = np.zeros(len(frequency_array))
    return {"plus": Zp, "cross": cross}

In [ ]:
#### Define the Likelihood according to what bilby likes
class My_Likelihood(bilby.Likelihood):

    def __init__(self, interferometers, waveform_generator, priors=None):

        super(My_Likelihood, self).__init__(dict())
        self.interferometers = interferometers[0]
        self.waveform_generator = waveform_generator
        self.priors = priors

    def priors(self):
        return self.priors

    def log_likelihood(self):

        waveform = self.waveform_generator.frequency_domain_strain(self.parameters)
        residual = self.interferometers.frequency_domain_strain - \
                    self.interferometers.get_detector_response(waveform, self.parameters)
        psd = self.interferometers.power_spectral_density_array
        duration = self.waveform_generator.duration

        log_l = -2.0 / duration * np.sum((np.conj(residual)*residual) / psd)
        
        return log_l.real

    def noise_log_likelihood(self):

        noise = self.interferometers.frequency_domain_strain
        psd = self.interferometers.power_spectral_density_array
        duration = self.waveform_generator.duration

        log_l = -2.0 / duration * np.sum(np.abs(noise)**2 / psd)
        
        return log_l.real

In [ ]:
#### Define the sampling frequency and the data duration
sampling_frequency = H1.sampling_frequency
duration = H1.duration


#### Call the waveform_generator to create our waveform model.
waveform = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=echo
)

In [ ]:
prior = bilby.core.prior.PriorDict()
prior['R'] = bilby.core.prior.Uniform(name='Reflection', minimum=0.0,maximum=1.0)
prior['d'] = bilby.core.prior.Uniform(name='Compactness', minimum=0.0, maximum=70.0)
prior['M'] = bilby.core.prior.Uniform(name="Mass", minimum=10, maximum=80, unit="$M_{sun}$")
prior['D'] = bilby.core.prior.Uniform(name="Luminosity Distance", minimum=0.1, maximum=1.0, unit="$Gpc$")
prior['wr'] =  0.3737 
prior['wi'] =  -0.08896 
prior['alpha'] =  0.2161 
prior['t_0'] =  -0.001 
prior['phi'] =  0.0
## Specifying the parameters of antenna pattern
prior['ra'] = 2.19432
prior['dec'] = -1.2232
prior['psi'] = 0.532268
prior['geocent_time'] = time_of_event

In [ ]:
#### Instantiate the Likelihood
likelihood = My_Likelihood(interferometers=[H1], waveform_generator=waveform, priors=prior)

In [ ]:
#### launch sampler
result2 = bilby.core.sampler.run_sampler(
    likelihood,
    prior,
    sampler="dynesty",
    npoints=500,
    walks=5,
    nact=3,
    outdir="GW150914_search",
    label="ECHO_search",
    dlogz=0.01
)


#### This will automatically show the signal-to-noise Bayes factor
#### Plot the corner plot
result.plot_corner()

In [ ]:
#### Plot the recovered signal
idxs = H1.strain_data.frequency_mask  # This is a boolean mask of the frequencies which we'll use in the analysis
plt.figure(figsize=(10, 6))
plt.loglog(H1.frequency_array[idxs], 
          H1.amplitude_spectral_density_array[idxs], label='H1 ASD', alpha=0.5, color='blue')
plt.loglog(waveform.frequency_array[idxs], np.sqrt(waveform.frequency_array[idxs])*
           np.abs(waveform.frequency_domain_strain()['plus'][idxs]), label='Best-Fit', color='tab:red')
plt.fill_between(waveform.frequency_array[idxs], np.sqrt(waveform.frequency_array[idxs])*
                 np.abs(waveform.frequency_domain_source_model(waveform.frequency_array, 0.92-0.04, 35.88-1.78, 15.83-0.52, 
                                                               0.78-0.15, 0.3737, -0.08896, 0.2161, -0.001, 0)['plus'])[idxs], 
                 np.sqrt(waveform.frequency_array[idxs])*
                 np.abs(waveform.frequency_domain_source_model(waveform.frequency_array, 0.92+0.03, 35.88+1.23, 15.83+0.81, 
                                                               0.78+0.14, 0.3737, -0.08896, 0.2161, -0.001, 0)['plus'])[idxs],
                 alpha=0.6, color='tab:green')

plt.xlabel('Frequency [Hz]')
plt.legend(framealpha=0.6)
plt.ylabel("Strain Amplitude")
plt.title("Best-Fit template")
plt.grid(True, which="both", ls=":")
plt.tight_layout()
#plt.savefig("./Freq_domain_bestfit_vs_psd.png", dpi=300)
plt.show()

In [ ]:
#### Calculate the SNR
sig = (1/(2*np.pi))*np.sqrt(waveform.frequency_array[idxs])*np.abs(waveform.frequency_domain_strain()['plus'][idxs])
snr = np.sqrt(4/H1.duration * np.sum((likelihood.interferometers[0].frequency_domain_strain[idxs]*np.conj(sig))/
                               likelihood.interferometers[0].power_spectral_density_array[idxs]).real)

print("The SNR:", snr)